In [1]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset

import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

/home/hice1/skaviles3/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/storage/ice1/2/2/skaviles3/data_merge.csv')
df.head()

,Unnamed: 0,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin,category
0,0,http://www.forbes.com/sites/steveschaefer/2016...,https://web.archive.org/web/2016120719id_/http...,Biotech Stocks Smacked After Trump Boasts He'l...,1970-01-01 00:33:36.120719,The post-election rebound in biotech and healt...,It turns out President Trump may be just as bi...,32.052632,0.894737,1.315789,medium,medium,abstractive,technology
1,1,http://www.9news.com.au/national/2015/08/21/15...,https://web.archive.org/web/2015082119id_/http...,Joe Hockey confirms GST will apply to online p...,1970-01-01 00:33:35.082119,Treasurer Joe Hockey has announced a ten perce...,Treasurer Joe Hockey has announced a ten perce...,10.350000,1.000000,20.000000,low,high,extractive,technology
2,2,http://www.foxnews.com/tech/2014/04/14/review-...,https://web.archive.org/web/2014041419id_/http...,Review: Siri-like Cortana fills Windows phone gap,1970-01-01 00:33:34.041419,Microsoft corporate vice president Joe Belfior...,"With the new Cortana virtual assistant, Window...",59.217391,1.000000,23.000000,high,high,extractive,technology
3,3,http://www.theguardian.com/technology/2014/nov...,https://web.archive.org/web/2014112119id_/http...,Now e-cigarettes can give you malware,1970-01-01 00:33:34.112119,E-cigarettes may be better for your health tha...,"Better for your lungs, worse for your hard dri...",22.041667,0.750000,1.750000,medium,low,mixed,technology
4,4,http://www.foxnews.com/scitech/2010/09/29/chin...,https://web.archive.org/web/2010092919id_/http...,China's Super Train Trounces Speed Records,1970-01-01 00:33:30.092919,258 miles per hour. That's how fast China's la...,A new Chinese high-speed train broke a world s...,11.111111,0.925926,5.296296,low,medium,mixed,technology


In [3]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Preprocess the input and output text
def preprocess_function(examples):
    inputs = tokenizer(examples['text'], max_length=1024, truncation=True, padding='max_length')
    labels = tokenizer(examples['summary'], max_length=128, truncation=True, padding='max_length')

    # Set labels for training
    inputs['labels'] = labels['input_ids']
    return inputs

In [4]:
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df[['text', 'summary']])

# Tokenize the dataset
tokenized_datasets = hf_dataset.map(preprocess_function, batched=True, remove_columns=["text", "summary"])


Map: 100%|██████████| 53978/53978 [02:26<00:00, 367.63 examples/s]


In [5]:
from sklearn.model_selection import train_test_split
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [6]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Define training arguments
training_args = TrainingArguments(
    output_dir="/storage/ice1/2/2/skaviles3/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
#     logging_dir='/storage/ice1/6/4/tchavan3/logs',
#     logging_steps=10,
#     report_to="none",  # Disable unnecessary logging reports
    
)


In [6]:
from evaluate import load
# Load the ROUGE metric
import evaluate
metric = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels=eval_pred # obtaining predictions and true labels
    
    # decoding predictions
    decoded_preds=tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # obtaining the true labels tokens, while eliminating any possible masked token (i.e: label=-100)
    labels=np.where(labels!=-100, labels, tokenizer.pad_token_id)
    decoded_labels=tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # rouge expects a newline after each sentence
    decoded_preds=['\n'.join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels=['\n'.join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # computing rouge score
    result=metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result={key: value.mid.fmeasure*100 for key, value in result.items()} # extracting some results
    
    # add mean-genrated length
    prediction_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in predictions]
    result['gen_len']=np.mean(prediction_lens)
    return {k: round(v,4) for k,v in result.items()}

In [18]:
import torch
torch.cuda.empty_cache()

In [8]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 20% data for evaluation
    compute_metrics=compute_metrics,
)

In [37]:
import torch
torch.cuda.empty_cache()

In [38]:
torch.cuda.empty_cache()

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 20% data for evaluation
)

# Train the model with a progress bar (tqdm is automatically included)
# trainer.train()
trainer.train(resume_from_checkpoint='/storage/ice1/2/2/skaviles3/results/checkpoint-2500')


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss
1,0.363200,0.340811
2,0.315100,0.333827
3,0.289000,0.332534



Training Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss


Training Epochs:   0%|          | 0/3 [00:21<?, ?it/s]


KeyboardInterrupt: 

In [17]:
path = "/storage/ice1/2/2/skaviles3/results/checkpoint-8097"
model = BartForConditionalGeneration.from_pretrained(path)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [19]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/technology_test.csv')

In [21]:
index=2
display(test_df[['text','summary']].iloc[index:index+1])
test_df['text'][index]

,text,summary
2,Tim Cook thinks he knows how to put $59.7 bill...,Tim Cook thinks he knows how to put $59.7 bill...


'Tim Cook thinks he knows how to put $59.7 billion to good use\n\nOne of the things that’s keeping Apple’s AAPL market cap from overtaking Exxon Mobil’s XOM — besides Steve Jobs’ health problems and the world’s unquenchable thirst for petroleum products — is the fear that the company will do something stupid with the nearly $60 billion in cash and marketable securities that seems to be burning a hole in Wall Street’s pocket.\n\nThe Street has made it abundantly clear what it thinks Apple should do with that cash: Declare a dividend or launch a stock repurchase program or both — anything to drive up the value of institutional investors’ Apple holdings.\n\nWhat the analysts who work for those institutions fear is that the company will go on a buying spree and start gobbling up the wrong kind of company — like Facebook, Adobe ADBE , Yahoo YHOO , Disney DIS or Sony SNE . Those are some of the acquisition candidates that were floated in the business press last October after Jobs was asked w

In [22]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []


In [23]:
for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

100%|██████████| 5448/5448 [51:12<00:00,  1.77it/s]  


In [24]:
average_scores_tech = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_tech

{'rouge1': 0.40448427724852554,
 'rouge2': 0.2865060241643397,
 'rougeL': 0.3597730412610299}

In [25]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/architecture_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 419/419 [03:51<00:00,  1.81it/s]


{'rouge1': 0.37809312413609153,
 'rouge2': 0.2530375844640264,
 'rougeL': 0.3283348249313546}

In [26]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/food_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 1482/1482 [14:10<00:00,  1.74it/s]


{'rouge1': 0.3492646792478155,
 'rouge2': 0.23684801467857072,
 'rougeL': 0.3073993441763635}

In [27]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/sports_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

  2%|▏         | 131/6163 [01:24<1:04:33,  1.56it/s]


KeyboardInterrupt: 

In [ ]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/entertainment_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

In [9]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
# Define training arguments
training_args = TrainingArguments(
    output_dir="/storage/ice1/2/2/skaviles3/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
#     logging_dir='/storage/ice1/6/4/tchavan3/logs',
#     logging_steps=10,
#     report_to="none",  # Disable unnecessary logging reports
    
)


In [ ]:
# Initialize the Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 20% data for evaluation
)

# Train the model with a progress bar (tqdm is automatically included)
# trainer.train()
trainer.train(resume_from_checkpoint='/storage/ice1/2/2/skaviles3/results/checkpoint-8097')


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss
4,0.295400,0.272686
5,0.276500,0.271287



Training Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss


Training Epochs:   0%|          | 0/5 [00:49<?, ?it/s]


KeyboardInterrupt: 

# BART injected model evaluation

In [13]:
from rouge_score import rouge_scorer

test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/technology_test.csv')

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_tech = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_tech

100%|██████████| 5448/5448 [33:00<00:00,  2.75it/s]


{'rouge1': 0.4070902623642649,
 'rouge2': 0.2892701118525887,
 'rougeL': 0.36235780344345825}

In [14]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/sports_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 6163/6163 [36:55<00:00,  2.78it/s]


{'rouge1': 0.3831651536464214,
 'rouge2': 0.2509568424128598,
 'rougeL': 0.3301597491044689}

In [15]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/food_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 1482/1482 [09:11<00:00,  2.69it/s]


{'rouge1': 0.35844792246022766,
 'rouge2': 0.2439043821636203,
 'rougeL': 0.31600797529629693}

In [16]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/architecture_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 419/419 [02:32<00:00,  2.75it/s]


{'rouge1': 0.37207757497468597,
 'rouge2': 0.2431804538260892,
 'rougeL': 0.3216438117869889}

In [17]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/entertainment_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 6668/6668 [40:20<00:00,  2.75it/s] 


{'rouge1': 0.40024860762551817,
 'rouge2': 0.28355068747929124,
 'rougeL': 0.3558658323663727}

# BART Baseline Evaluation

In [20]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').to(device)

In [21]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/entertainment_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 6668/6668 [1:31:04<00:00,  1.22it/s]


{'rouge1': 0.25456955875129933,
 'rouge2': 0.16404162263024455,
 'rougeL': 0.2139045876625417}

In [22]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/architecture_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 419/419 [05:42<00:00,  1.22it/s]


{'rouge1': 0.24576866517263188,
 'rouge2': 0.1547942113323618,
 'rougeL': 0.2059425862827461}

In [23]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/food_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 1482/1482 [20:15<00:00,  1.22it/s]


{'rouge1': 0.2333310144247373,
 'rouge2': 0.14703136749240425,
 'rougeL': 0.1979435435894947}

In [24]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/sports_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_arch = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_arch

100%|██████████| 6163/6163 [1:24:47<00:00,  1.21it/s]


{'rouge1': 0.23407144187032236,
 'rouge2': 0.14129025812317933,
 'rougeL': 0.19308868959917938}

In [25]:
test_df = pd.read_csv('/storage/ice1/2/2/skaviles3/technology_test.csv')
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

for idx, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    inputs = tokenizer(row['text'], return_tensors="pt", truncation=True, max_length=1024).to(device)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    score = scorer.score(row['summary'], predicted_summary)
    rouge_scores.append(score)

average_scores_tech = {metric: sum([score[metric].fmeasure for score in rouge_scores]) / len(rouge_scores) for metric in rouge_scores[0]}
average_scores_tech

100%|██████████| 5448/5448 [1:14:13<00:00,  1.22it/s]


{'rouge1': 0.25317686616376717,
 'rouge2': 0.16564610536302368,
 'rougeL': 0.21556226523124777}